<a href="https://colab.research.google.com/github/fred-creator-creat/projeto-redes-sociais-grafo/blob/main/Projeto_Redes_Sociais_Grafo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
import os

# Aqui o código aponta para os nomes criados no "cofre"
os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USER')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

print("Teste de conexão: As credenciais foram carregadas do cofre com sucesso!")

Teste de conexão: As credenciais foram carregadas do cofre com sucesso!


In [2]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.3/325.3 kB 6.5 MB/s eta 0:00:00


In [3]:
from neo4j import GraphDatabase
from google.colab import userdata

# Buscando as credenciais do Aura criados no cofre
uri = userdata.get('AURA_URI')
user = userdata.get('AURA_USER')
password = userdata.get('AURA_PASSWORD')

# Função para testar a conexão
def testar_conexao():
    try:
        driver = GraphDatabase.driver(uri, auth=(user, password))
        with driver.session() as session:
            # Uma query simples para pedir a versão do servidor
            resultado = session.run("RETURN 'Conexão com o Aura realizada com sucesso!' AS msg").single()
            print(resultado["msg"])
        driver.close()
    except Exception as e:
        print(f"Erro ao conectar: {e}")

testar_conexao()

Conexão com o Aura realizada com sucesso!


In [4]:
# Buscando bases de dados de redes sociais
!kaggle datasets list -s "social network"

ref                                                            title                                                 size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  ----------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
d4rklucif3r/social-network-ads                                 Social Network Ads                                    1497  2021-06-07 14:31:39.520000          20955        117  0.9411765        
csanhueza/the-marvel-universe-social-network                   The Marvel Universe Social Network                 2855190  2017-01-28 11:28:49.053000          28917        248  0.9705882        
rakeshrau/social-network-ads                                   Social Network Ads                                    3349  2017-08-06 11:52:21.617000          78614        220  0.5294118        
shub99/social-network-ads

In [10]:
from neo4j import GraphDatabase
from google.colab import userdata

# Buscando as chaves do cofre para o driver não ser "esquecido"
uri = userdata.get('AURA_URI')
user = userdata.get('AURA_USER')
password = userdata.get('AURA_PASSWORD')

# Criando o driver e enviando os dados
driver = GraphDatabase.driver(uri, auth=(user, password))

# Criando a sessão para enviar o desenho para o AuraDB
with driver.session() as session:
    # COLANDO O CÓDIGO CYPHER DO ARROWS ENTRE AS TRÊS ASPAS ABAIXO:
    query = """
    MERGE (n0:User {name: "Fred Cavalheiro", username: "@fredcav"})-[:FOLLOWS]->(n0)-[:POSTED]->(n1:Post {content: "Aprendendo Neo4j", likes: 100})<-[:LIKED]-(n0)
    MERGE (n1)-[:ON_DATE]->(:Date {date: "2026-02-21"})
    """
    session.run(query)
    print("O seu desenho do Arrows agora é REALIDADE dentro do AuraDB!")

driver.close()

O seu desenho do Arrows agora é REALIDADE dentro do AuraDB!


In [9]:
from neo4j import GraphDatabase
from google.colab import userdata

# Abrindo a conexão
uri = userdata.get('AURA_URI')
user = userdata.get('AURA_USER')
password = userdata.get('AURA_PASSWORD')
driver = GraphDatabase.driver(uri, auth=(user, password))

with driver.session() as session:

    query = """
    MERGE (ana:User {name: "Ana Silva", username: "@ana_silva"})
    MERGE (bruno:User {name: "Bruno Costa", username: "@brunoc"})
    MERGE (carla:User {name: "Carla Souza", username: "@carla_s"})

    WITH ana, bruno, carla

    // Conectando com o usuário Fred que já existe
    MATCH (fred:User {username: "@fredcav"})

    // Criando o ciclo de amizades (Fred segue Ana, que segue Bruno...)
    MERGE (fred)-[:FOLLOWS]->(ana)
    MERGE (ana)-[:FOLLOWS]->(bruno)
    MERGE (bruno)-[:FOLLOWS]->(carla)
    MERGE (carla)-[:FOLLOWS]->(fred)

    // Criando um post e uma curtida
    MERGE (p2:Post {content: "O curso de Neo4j está incrível!", likes: 50})
    MERGE (ana)-[:POSTED]->(p2)
    MERGE (bruno)-[:LIKED]->(p2)
    """
    session.run(query)
    print("Rede social expandida com sucesso!")

Rede social expandida com sucesso!


In [12]:
# Garante que o driver está ativo
if 'driver' not in locals() or driver._closed:
    from neo4j import GraphDatabase
    from google.colab import userdata
    uri = userdata.get('AURA_URI')
    user = userdata.get('AURA_USER')
    password = userdata.get('AURA_PASSWORD')
    driver = GraphDatabase.driver(uri, auth=(user, password))

with driver.session() as session:
    # Pergunta: Quem meus amigos seguem que eu ainda não sigo?
    query = """
    MATCH (fred:User {username: "@fredcav"})-[:FOLLOWS]->(amigo)-[:FOLLOWS]->(sugestao)
    WHERE NOT (fred)-[:FOLLOWS]->(sugestao) AND sugestao <> fred
    RETURN sugestao.name AS Nome, sugestao.username AS User
    """
    result = session.run(query)

    print("--- SUGESTÕES DE AMIZADE PARA VOCÊ ---")
    encontrou = False
    for record in result:
        encontrou = True
        print(f"Sugestão: {record['Nome']} ({record['User']})")

    if not encontrou:
        print("Nenhuma recomendação nova encontrada.")

--- SUGESTÕES DE AMIZADE PARA VOCÊ ---
Sugestão: Bruno Costa (@brunoc)


In [13]:
# Garante que o driver está ativo
if 'driver' not in locals() or driver._closed:
    from neo4j import GraphDatabase
    from google.colab import userdata
    uri = userdata.get('AURA_URI')
    user = userdata.get('AURA_USER')
    password = userdata.get('AURA_PASSWORD')
    driver = GraphDatabase.driver(uri, auth=(user, password))

with driver.session() as session:
    # Pergunta: Qual o caminho mais curto entre Fred e Carla?
    query = """
    MATCH (fred:User {username: "@fredcav"}), (carla:User {username: "@carla_s"}),
    p = shortestPath((fred)-[:FOLLOWS*..10]->(carla))
    RETURN p
    """
    result = session.run(query)

    print("--- ANÁLISE DE CONEXÃO (CAMINHO MAIS CURTO) ---")
    for record in result:
        path = record['p']
        # Mostra o caminho: Fred -> Ana -> Bruno -> Carla
        nodes = [node['name'] for node in path.nodes]
        print(" -> ".join(nodes))

--- ANÁLISE DE CONEXÃO (CAMINHO MAIS CURTO) ---
Fred Cavalheiro -> Ana Silva -> Bruno Costa -> Carla Souza


In [14]:
!pip install pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.4 MB/s eta 0:00:00


In [16]:
from pyvis.network import Network
from neo4j import GraphDatabase
from google.colab import userdata
import IPython

# 1. Conecta ao banco
uri = userdata.get('AURA_URI')
user = userdata.get('AURA_USER')
pwd = userdata.get('AURA_PASSWORD')
driver = GraphDatabase.driver(uri, auth=(user, pwd))

def draw_graph():
    # Adicionado cdn_resources='remote' para funcionar no Chrome/Colab
    net = Network(notebook=True, directed=True, height="500px", width="100%", bgcolor="#222222", font_color="white", cdn_resources='remote')

    with driver.session() as session:
        result = session.run("MATCH (n)-[r]->(m) RETURN n, r, m")

        for record in result:
            n, r, m = record['n'], record['r'], record['m']

            # Cores: Usuário (Vermelho), Post (Verde), Data (Amarelo)
            n_color = "#FF4B4B" if "User" in n.labels else "#4BFF4B" if "Post" in n.labels else "#FFFF4B"
            m_color = "#FF4B4B" if "User" in m.labels else "#4BFF4B" if "Post" in m.labels else "#FFFF4B"

            # Pegando os rótulos corretos
            n_label = n.get('name') or n.get('content') or n.get('date')
            m_label = m.get('name') or m.get('content') or m.get('date')

            net.add_node(n.element_id, label=str(n_label), color=n_color)
            net.add_node(m.element_id, label=str(m_label), color=m_color)
            net.add_edge(n.element_id, m.element_id, title=r.type, label=r.type)

    net.show("graph.html")
    # Força a exibição do arquivo gerado dentro do Colab
    return IPython.display.HTML("graph.html")

# Executa e mostra
draw_graph()

graph.html


In [17]:
from neo4j import GraphDatabase
from google.colab import userdata

# Garante a conexão ativa
uri = userdata.get('AURA_URI')
user = userdata.get('AURA_USER')
password = userdata.get('AURA_PASSWORD')
driver = GraphDatabase.driver(uri, auth=(user, password))

with driver.session() as session:
    # Pergunta: Qual o post mais curtido e quem é o autor?
    query = """
    MATCH (user:User)-[:POSTED]->(post:Post)
    RETURN user.name AS Autor, post.content AS Conteudo, post.likes AS Curtidas
    ORDER BY post.likes DESC
    LIMIT 1
    """
    result = session.run(query)
    print("--- POST MAIS POPULAR DA REDE ---")
    for record in result:
        print(f"Autor: {record['Autor']}")
        print(f"Post: {record['Conteudo']}")
        print(f"Total de Curtidas: {record['Curtidas']}")

--- POST MAIS POPULAR DA REDE ---
Autor: Fred Cavalheiro
Post: Aprendendo Neo4j
Total de Curtidas: 100


In [19]:
from neo4j import GraphDatabase
from google.colab import userdata

# Garante a conexão ativa
uri = userdata.get('AURA_URI')
user = userdata.get('AURA_USER')
password = userdata.get('AURA_PASSWORD')
driver = GraphDatabase.driver(uri, auth=(user, password))

with driver.session() as session:
    # O comando PROFILE analisa o custo técnico da consulta
    query = """
    PROFILE MATCH (n:User {username: "@fredcav"})-[:FOLLOWS]->(amigo)
    RETURN amigo.name
    """
    result = session.run(query)
    summary = result.consume()

    print("--- RELATÓRIO DE PERFORMANCE PROFISSIONAL ---")
    print(f"Tempo de execução: {summary.result_available_after} ms")
    if summary.profile:
        # Pega o total de 'hits' (esforço do banco)
        db_hits = summary.profile.get('db_hits', 0)
        print(f"Esforço do banco (db_hits): {db_hits}")
    else:
        print("Perfil de execução não disponível.")

    print("\nConsulta finalizada com sucesso seguindo as diretrizes do desafio.")

driver.close()

--- RELATÓRIO DE PERFORMANCE PROFISSIONAL ---
Tempo de execução: 1 ms
Esforço do banco (db_hits): 0

Consulta finalizada com sucesso seguindo as diretrizes do desafio.
